In [153]:
module Oct

using StaticArrays
import StaticArrays: size, getindex

immutable Partition{N, T, L} <: StaticArray{T, N}
    data::NTuple{L, T}
end
@generated function Partition{L, T}(x::NTuple{L, T})
    @assert log2(L) == round(log2(L)) "L must equal 2^N for integer N"
    N = Int(log2(L))
    :(Partition{$N, T, L}(x))
end

@generated size{N, T, L}(::Type{Partition{N, T, L}}) = Expr(:tuple, [2 for i in 1:N]...)
getindex(b::Partition, i::Integer) = b.data[i]

type Cell{Data, N, T, L}
    origin::SVector{N, T}
    widths::SVector{N, T}
    data::Data
    divisions::SVector{N, T}
    children::Nullable{Partition{N, Cell{Data, N, T}, L}}
end

@generated function Cell{Data, N, T}(origin::SVector{N, T}, widths::SVector{N, T}, data::Data=nothing)
    L = 2^N
    :(Cell{Data, N, T, $L}(origin, widths, data, widths / 2, Nullable{Partition{N, T, $L}}()))
end

isleaf(cell::Cell) = !isnull(cell.children)

split_data(cell, offset) = data
split_data(cell::Cell{String}, offset::Tuple) = cell.data * "|$(offset)"

getindex(cell::Cell, I) = getindex(get(cell.children), I)
getindex(cell::Cell, I...) = getindex(get(cell.children), I...)

@generated function split!(cell::Cell)
    split!_impl(cell)
end

function split!_impl{Data, N, T, L}(::Type{Cell{Data, N, T, L}})
    children = Array{Expr, N}([2 for i in 1:N]...)
    for I in CartesianRange(tuple([2 for i in 1:N]...))
        offset = Expr(:call, :SVector, (0.5 * (SVector(I.I) - 1))...)
        children[I] = :(Cell(cell.origin + $(offset) .* cell.widths, 
                             cell.widths ./ 2, 
                             split_data(cell, $(I.I))))
    end
    
    quote
        @assert !isleaf(cell)
        cell.children = Nullable(Partition($(Expr(:tuple, children...))))
    end
end



end
    

Oct

In [154]:
cell = Oct.Cell(SVector(0., 0), SVector(1., 1), "")

Oct.Cell{String,2,Float64,4}([0.0,0.0],[1.0,1.0],"",[0.5,0.5],Nullable{Oct.Partition{2,Oct.Cell{String,2,Float64,L},4}}())

In [155]:
cell.data

""

In [156]:
Oct.split!(cell)

Nullable{Oct.Partition{2,Oct.Cell{String,2,Float64,4},4}}(Oct.Cell{String,2,Float64,4}[Oct.Cell{String,2,Float64,4}([0.0,0.0],[0.5,0.5],"|(1,1)",[0.25,0.25],#NULL) Oct.Cell{String,2,Float64,4}([0.0,0.5],[0.5,0.5],"|(1,2)",[0.25,0.25],#NULL); Oct.Cell{String,2,Float64,4}([0.5,0.0],[0.5,0.5],"|(2,1)",[0.25,0.25],#NULL) Oct.Cell{String,2,Float64,4}([0.5,0.5],[0.5,0.5],"|(2,2)",[0.25,0.25],#NULL)])

In [157]:
cell[1,2].data

"|(1,2)"

In [158]:
Oct.split!(cell[1,2])

Nullable{Oct.Partition{2,Oct.Cell{String,2,Float64,4},4}}(Oct.Cell{String,2,Float64,4}[Oct.Cell{String,2,Float64,4}([0.0,0.5],[0.25,0.25],"|(1,2)|(1,1)",[0.125,0.125],#NULL) Oct.Cell{String,2,Float64,4}([0.0,0.75],[0.25,0.25],"|(1,2)|(1,2)",[0.125,0.125],#NULL); Oct.Cell{String,2,Float64,4}([0.25,0.5],[0.25,0.25],"|(1,2)|(2,1)",[0.125,0.125],#NULL) Oct.Cell{String,2,Float64,4}([0.25,0.75],[0.25,0.25],"|(1,2)|(2,2)",[0.125,0.125],#NULL)])

In [159]:
cell[1,2][1,1].origin

2-element StaticArrays.SVector{2,Float64}:
 0.0
 0.5

In [160]:
cell[1,2][1,1].data

"|(1,2)|(1,1)"